In [141]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score,learning_curve, GroupKFold,GridSearchCV
from sklearn.metrics import confusion_matrix



In [142]:
df =  pd.read_csv('../2017_07_07/TADPOLE_D1_D2.csv',low_memory=False)

In [143]:
#Name of the columns
df.head(0)


,RID,PTID,VISCODE,SITE,D1,D2,COLPROT,ORIGPROT,EXAMDATE,DX_bl,DXCHANGE,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,FDG,PIB,AV45,CDRSB,ADAS11,ADAS13,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,FAQ,MOCA,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal,FLDSTRENG,FSVERSION,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,ICV,DX,EXAMDATE_bl,CDRSB_bl,ADAS11_bl,ADAS13_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,FAQ_bl,FLDSTRENG_bl,FSVERSION_bl,Ventricles_bl,Hippocampus_bl,WholeBrain_bl,Entorhinal_bl,Fusiform_bl,MidTemp_bl,ICV_bl,MOCA_bl,EcogPtMem_bl,EcogPtLang_bl,EcogPtVisspat_bl,EcogPtPlan_bl,EcogPtOrgan_bl,EcogPtDivatt_bl,EcogPtTotal_bl,EcogSPMem_bl,EcogSPLang_bl,EcogSPVisspat_bl,EcogSPPlan_bl,EcogSPOrgan_bl,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp,EXAMDATE_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,VERSION_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,FLDSTRENG_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,LONISID_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,LONIUID_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,IMAGEUID_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,RUNDATE_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,STATUS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP1_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP2_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP3_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP4_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP5_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP6_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP7_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP8_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,OVERALLQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,TEMPQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,FRONTQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,PARQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,INSULAQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,OCCQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BGQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,CWMQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,VENTQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST100SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST101SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST102CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST102SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST102TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST102TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST103CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST103SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST103TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST103TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST104CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST104SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST104TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST104TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST105CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST105SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST105TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST105TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST106CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST106SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST106TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST106TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST107CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST107SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST107TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST107TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST108CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST108SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST108TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST108TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST109CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST109SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST109TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST109TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST10CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST110CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST110SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST110TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST110TS_UCSFFSL_02_01

In [144]:
#Do some test on the data

print(df['DX'].head())
print(df['DX_bl'].head())
print(df['EXAMDATE_BAIPETNMRC_09_12_16'].unique())
print(df['PTETHCAT'].head(1))

0          NL
1    Dementia
2    Dementia
3    Dementia
4    Dementia
Name: DX, dtype: object
0    CN
1    AD
2    AD
3    AD
4    AD
Name: DX_bl, dtype: object
['2005-09-22' '2006-03-13' '2006-09-12' ... '2016-04-21' '2016-04-12'
 '2016-04-27']
0    Not Hisp/Latino
Name: PTETHCAT, dtype: object


In [145]:
pd.set_option('display.max_columns', None)

#important columns to convert to int: EXAMDATE	DX_bl	DXCHANGE	AGE	PTGENDER	PTEDUCAT	PTETHCAT	PTRACCAT	PTMARRY	

In [146]:
print(df['DXCHANGE'].head(1))
'1=Stable:NL to NL, 2=Stable:MCI to MCI, 3=Stable:AD to AD, ' 
'4=Conv:NL to MCI, 5=Conv:MCI to AD, 6=Conv:NL to AD, ' 
'7=Rev:MCI to NL, 8=Rev:AD to MCI, 9=Rev:AD to NL, -1=Not available'

0    1.0
Name: DXCHANGE, dtype: float64


'7=Rev:MCI to NL, 8=Rev:AD to MCI, 9=Rev:AD to NL, -1=Not available'

In [147]:
data = df
print(data.shape)

#Convert to numeric
data['PTGENDER']= data['PTGENDER'].replace(['Male','Female'],[1,0])
data['APOE4']= data['APOE4'].astype(float)
data['AGE']= data['AGE'].astype(float)
data['PTETHCAT']= data['PTETHCAT'].replace(['Hisp/Latino','Not Hisp/Latino','Unknown'],[2,1,0]).astype(int)
data['PTMARRY']= data['PTMARRY'].replace(['Married','Widowed','Divorced','Never married','Unknown'],[0,1,2,3,4]).astype(int)
data['PTRACCAT'] = data['PTRACCAT'].replace(['White','More than one','Black','Asian', 'Am Indian/Alaskan','Hawaiian/Other PI', 'Unknown'],[0,1,2,3,4,5,6]).astype(int)
data['DX_bl'] = data['DX_bl'].replace(['CN' ,'AD' ,'LMCI' ,'EMCI' ,'SMC'],['0','1','2','3','4']).astype(int)
df['DX'] = df['DX'].replace(['NL' ,'Dementia', 'MCI', 'NL to MCI', 'MCI to Dementia', 'MCI to NL','Dementia to MCI','nan' ,'NL to Dementia'],['1','3','4','5','6','7','8','9','10'])
df['DX']=df['DX'].fillna(0)

#Print some data to check if it is correct
print(data['DX_bl'].head())
print(data['DX'].unique())

# drop the columns in data they ate objet type
data = data.select_dtypes(exclude=['object'])
print(data.shape)


(12741, 1907)
0    0
1    1
2    1
3    1
4    1
Name: DX_bl, dtype: int32
['1' '3' '4' '5' '6' '7' '8' 0 '10']
(12741, 85)


In [148]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12741 entries, 0 to 12740
Data columns (total 85 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   RID                       12741 non-null  int64  
 1   SITE                      12741 non-null  int64  
 2   D1                        12741 non-null  int64  
 3   D2                        12741 non-null  int64  
 4   DX_bl                     12741 non-null  int32  
 5   DXCHANGE                  8892 non-null   float64
 6   AGE                       12741 non-null  float64
 7   PTGENDER                  12741 non-null  int64  
 8   PTEDUCAT                  12741 non-null  int64  
 9   PTETHCAT                  12741 non-null  int32  
 10  PTRACCAT                  12741 non-null  int32  
 11  PTMARRY                   12741 non-null  int32  
 12  APOE4                     12729 non-null  float64
 13  FDG                       3352 non-null   float64
 14  PIB   

In [149]:
# Algorithme des k plus proches voisins
model = KNeighborsClassifier()
y = data['DXCHANGE']
X = data.drop('DXCHANGE', axis=1)
y = np.nan_to_num(y)
X= np.nan_to_num(X)


In [150]:

#Split the data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

model.fit(X_train,y_train)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

0.7055533751962323
0.5810121616320125


In [151]:
#Take the best parameters for the model
param_grid = {'n_neighbors': np.arange(1, 25),'metric': ['euclidean', 'manhattan']}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid.fit(X_train, y_train)

C:\Users\antoi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# Print the tuned parameters and score
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_estimator_)

model = grid.best_estimator_
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

In [ ]:
#View the error of the model
confusion_matrix(y_test, model.predict(X_test))


In [ ]:
#the learning curve
N, train_score,val_score = learning_curve(model,X_train,y_train,train_sizes = np.linspace(0.2,1.0,5),cv=5)

In [ ]:
#Plot the learning curve
print(N)
plt.plot(N,train_score.mean(axis=1), label='train')
plt.plot(N,val_score.mean(axis=1), label='validation')
plt.xlabel('train_sizes')
plt.legend()


In [ ]:
#Chose the best split for the data
cv =  GroupKFold(5).get_n_splits(X,y,groups=X[:,5])
cross_val_score(model,X,y,cv=cv)

In [ ]:
def sick(model,PTGENDER=1,AGE=10,PTEDUCAT=4,APOE4=0):
    x= np.array([PTGENDER,AGE,PTEDUCAT,APOE4]).reshape(1,4)
    print(model.predict(x))
    print(model.predict_proba(x))